In [95]:
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from nni.algorithms.compression.pytorch.quantization import LsqQuantizer, QAT_Quantizer

import torch.nn as nn
import onnx
import onnx.numpy_helper
from math import ceil
### markus
import numpy as np
import PIL
import os
from re import L
import subprocess
from subprocess import DEVNULL, STDOUT
from xmlrpc.client import boolean
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
#%matplotlib notebook
import ProfileClass as prof


# imgSizeW= 68#1280
# imgSizeH= 18#720
# gen_img_input_dim_w = imgSizeW
# gen_img_input_dim_h = imgSizeH
# gen_img_input_channels = 3


In [96]:
# imgSizeW= 68#1280

# imgSizeH= 18#720
# gen_img_input_dim_w = imgSizeW
# gen_img_input_dim_h = imgSizeH
# gen_img_input_channels = 3
# #test input image
# test_input_data="../convertdemo/dataset/rand_3.jpg"
# #Paths
# quant_image_path = "../quantization_images"
# script_path = "../scripts"
# log_path = "../logs"
# network_path = "../convertdemo/network"
# #Files
# perform_script = "perform_r6.sh"
# parse_script = "parse_r1.sh"
# perform_log_file = "model_execution.log"
# parsed_log_file = "model_execution_parsed.log"
# model_name="mnist"


# Build the normal model

### Example Model 1

In [97]:
import math
import torch
import torch.nn as nn
from builder import *
base_factor = 32
class slice_reshape_operation(nn.Module):
    def __init__(self, op_name, C_in, C_out, img_w, img_h, expansion, stride):
        super(slice_reshape_operation, self).__init__()
        self.img_w = img_w*base_factor
        self.img_h = img_h*base_factor
        self.C_in = C_in*base_factor
        self.op = PRIMITIVES[op_name](self.C_in, C_out*base_factor, expansion, stride)
        
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.select(1,0)
        #x = x.select(2,0)
        x = torch.reshape(x,(batch_size, self.C_in,  self.img_h, self.img_w))
        #x = x.reshape(batch_size, self.C_in,  self.img_h, self.img_w)
        x = self.op(x)

        return x
        
        

### Example Model 2

In [98]:
class slice_reshape_operation_2(nn.Module):
    def __init__(self, op_name, C_in, C_out, img_w, img_h, expansion, stride):
        super(slice_reshape_operation_2, self).__init__()
        self.img_w = img_w*base_factor
        self.img_h = img_h*base_factor
        self.C_in = C_in*base_factor
        self.op1 = PRIMITIVES[op_name](self.C_in, C_out*base_factor, expansion, stride)
        self.op2 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op3 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.select(1,0)
        #x = x.select(2,0)
        x = torch.reshape(x,(batch_size, self.C_in,  self.img_h, self.img_w))
        #x = x.reshape(batch_size, self.C_in,  self.img_h, self.img_w)
        x = self.op1(x)
        x = self.op2(x)
        x = self.op3(x)
        return x

### Example Model 3

In [99]:
class slice_reshape_operation_3(nn.Module):
    def __init__(self, op_name, C_in, C_out, img_w, img_h, expansion, stride):
        super(slice_reshape_operation_3, self).__init__()
        self.img_w = img_w*base_factor
        self.img_h = img_h*base_factor
        self.C_in = C_in*base_factor
        self.op1 = PRIMITIVES[op_name](self.C_in, C_out*base_factor, expansion, stride)
        self.op2 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op3 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op4 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.select(1,0)
        #x = x.select(2,0)
        x = torch.reshape(x,(batch_size, self.C_in,  self.img_h, self.img_w))
        #x = x.reshape(batch_size, self.C_in,  self.img_h, self.img_w)
        x = self.op1(x)
        x = self.op2(x)
        x = self.op3(x)
        x = self.op4(x)
        return x

### Example Model 4

In [100]:
class slice_reshape_operation_4(nn.Module):
    def __init__(self, op_name, C_in, C_out, img_w, img_h, expansion, stride):
        super(slice_reshape_operation_4, self).__init__()
        self.img_w = img_w*base_factor
        self.img_h = img_h*base_factor
        self.C_in = C_in*base_factor
        self.op1 = PRIMITIVES[op_name](self.C_in, C_out*base_factor, expansion, stride)
        self.op2 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op3 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op4 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op5 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.select(1,0)
        #x = x.select(2,0)
        x = torch.reshape(x,(batch_size, self.C_in,  self.img_h, self.img_w))
        #x = x.reshape(batch_size, self.C_in,  self.img_h, self.img_w)
        x = self.op1(x)
        x = self.op2(x)
        x = self.op3(x)
        x = self.op4(x)
        x = self.op5(x)
        return x

### Example Model 5

In [101]:
class slice_reshape_operation_5(nn.Module):
    def __init__(self, op_name, C_in, C_out, img_w, img_h, expansion, stride):
        super(slice_reshape_operation_5, self).__init__()
        self.img_w = img_w*base_factor
        self.img_h = img_h*base_factor
        self.C_in = C_in*base_factor
        self.op1 = PRIMITIVES[op_name](self.C_in, C_out*base_factor, expansion, stride)
        self.op2 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op3 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op4 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op5 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
        self.op6 = PRIMITIVES[op_name](C_out*base_factor, C_out*base_factor, expansion, stride)
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.select(1,0)
        #x = x.select(2,0)
        x = torch.reshape(x,(batch_size, self.C_in,  self.img_h, self.img_w))
        #x = x.reshape(batch_size, self.C_in,  self.img_h, self.img_w)
        x = self.op1(x)
        x = self.op2(x)
        x = self.op3(x)
        x = self.op4(x)
        x = self.op5(x)
        x = self.op6(x)
        return x

In [102]:
op_name = "ir_k3_re"

C_in = 4
C_out = 4
img_w = 10
img_h = 10
expansion = 6
stride = 1

#generate input to test
input_x = torch.rand(1,3,base_factor*base_factor ,img_w*img_h*C_in*base_factor)

# print(input_x.shape)
# model = slice_reshape_operation_2(op_name, C_in, C_out, img_w, img_h, expansion, stride)

# y = model(input_x)

In [103]:
#model = slice_reshape_operation(op_name, C_in, C_out, img_w, img_h, expansion, stride)
#model = slice_reshape_operation_2(op_name, C_in, C_out, img_w, img_h, expansion, stride)
#model = slice_reshape_operation_3(op_name, C_in, C_out, img_w, img_h, expansion, stride)
#model = slice_reshape_operation_4(op_name, C_in, C_out, img_w, img_h, expansion, stride)
model = slice_reshape_operation_5(op_name, C_in, C_out, img_w, img_h, expansion, stride)

# Call the profile function
the class is:
- import ProfileClass as prof
- MP = prof.ModelProfile()

the function is:

- doProfiling(model, loop_run, gen_img_input_channels, gen_img_input_dim_h, gen_img_input_dim_w, debug=True) 

the parameters are:
- the model to be profiled
- number of runs to be profiled
- channels for images
- image height
- image width
- debugging enable

In [104]:
gen_img_input_channels = 3
gen_img_input_dim_h = base_factor*base_factor
gen_img_input_dim_w = img_w*img_h*C_in*base_factor

In [105]:
loop_run = '10'
MP = prof.ModelProfile()
[ProfileArray,status] = MP.doProfiling(model,
                 loop_run,
                 gen_img_input_channels,
                 gen_img_input_dim_h,
                 gen_img_input_dim_w,
                 debug=True)

accData=ProfileArray.values.tolist()

Creating ModelProfile Class...
------------- Exporting to onnx


Exported graph: graph(%input : Float(1, 3, 1024, 12800, strides=[39321600, 13107200, 12800, 1], requires_grad=0, device=cpu),
      %onnx::Conv_176 : Float(768, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_177 : Float(768, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_179 : Float(768, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_182 : Float(128, 768, 1, 1, strides=[768, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_183 : Float(128, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_185 : Float(768, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_188 : Float(768, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_191 : Float(128, 768, 1, 1, strides=[768, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_194 : Float(768, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_197 : Fl

vnn_mnist.c: In function ‘vnn_CreateMnist’:
vnn_mnist.c:146:29: warning: unused variable ‘data’ [-Wunused-variable]
  146 |     uint8_t *               data;
      |                             ^~~~
At top level:
vnn_mnist.c:94:17: warning: ‘load_data’ defined but not used [-Wunused-function]
   94 | static uint8_t* load_data
      |                 ^~~~~~~~~


  COMPILE /home/khadas/nbg_unify_mnist/main.c
  COMPILE /home/khadas/nbg_unify_mnist/vnn_post_process.c
make: Nothing to be done for 'all'.
>>>>> Compiling the model on Remote Khadas Done! <<<<<<<
>>>>> Copying test input image to Remote... <<<<<<<
>>>>> Copying test input image to Remote! <<<<<<<
>>>>> Executing the model on Remote Khadas... <<<<<<<
>>>>> Executing the model on Remote Khadas Done! <<<<<<<
------------- Performing the profiling done!
------------- Parsing the profiling results...
>>>>> Executing network on Khadas is saved in model_execution.log <<<<<<<
Parsing ../logs/model_execution.log....
------------- Parsing the profiling results done!
------------- auto_profile done!...


In [106]:
C0 = np.array(ProfileArray[0])
C1 = np.array(ProfileArray[1])
C2 = np.array(ProfileArray[2])
print('---------\n')
print(C0)
print('---------\n')
print(C1)
print('---------\n')
print(C2)
print('---------\n')
print(status)



---------

['Create Neural Network' 'Verify Graph' 'Run the 1 time' 'Run the 2 time'
 'Run the 3 time' 'Run the 4 time' 'Run the 5 time' 'Run the 6 time'
 'Run the 7 time' 'Run the 8 time' 'Run the 9 time' 'Run the 10 time'
 'Total   ' 'Average ']
---------

[' 91492us' ' 50566us' ' 1542983.00us' ' 1538697.00us' ' 1539738.00us'
 ' 1540626.00us' ' 1540729.00us' ' 1537278.00us' ' 1541310.00us'
 ' 1539103.00us' ' 1541940.00us' ' 1539051.00us' ' 15401659.00us'
 ' 1540165.88us']
---------

['91492' '50566' '1542983' '1538697' '1539738' '1540626' '1540729'
 '1537278' '1541310' '1539103' '1541940' '1539051' '15401659' '1540165']
---------

True
